# IMDB 영화 리뷰 감성 분석 - Conv1D

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
import warnings
warnings.filterwarnings('ignore')

In [2]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()
X_train.shape, X_test.shape, y_train.shape

17473536/17464789 [==============================] - 0s 0us/step


((25000,), (25000,), (25000,))

In [3]:
print('등장한 단어수', len(imdb.get_word_index()))
print('리뷰 최대 길이', max(len(s) for s in X_train))
print('리뷰 평균 길이', sum(map(len, X_train)) / len(X_train))

1654784/1641221 [==============================] - 0s 0us/step
등장한 단어수 88584
리뷰 최대 길이 2494
리뷰 평균 길이 238.71364


### Conv1D으로 IDBM 리뷰 감성 분석
- 단어 빈도수: 10,000 (88,584)
- 문장의 단어수: 500 (2,494)
- test data중 40%(10,000개)는 검증용으로

In [4]:
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [15]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [7]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
X_train.shape, X_test.shape, y_train.shape

((25000,), (25000,), (25000,))

In [8]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [9]:
from sklearn.model_selection import train_test_split
X_test, X_valid, y_test, y_valid = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.4, random_state=seed
)
X_test.shape, X_valid.shape, y_test.shape, y_valid.shape

((15000, 500), (10000, 500), (15000,), (10000,))

- Case 2) Conv1D x 2, MaxPooling1D x 2,
Dropout, GlobalMaxPooling1D
    - embedding dim: 100

In [17]:
model1 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_3 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                  

In [18]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [19]:
hist1 = model1.fit(
    X_train, y_train, epochs=30, batch_size=128, validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
194/196 [============================>.] - ETA: 0s - loss: 0.5301 - accuracy: 0.7132
Epoch 00001: val_loss improved from inf to 0.36458, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 4s 17ms/step - loss: 0.5288 - accuracy: 0.7143 - val_loss: 0.3646 - val_accuracy: 0.8359
Epoch 2/30
193/196 [============================>.] - ETA: 0s - loss: 0.2761 - accuracy: 0.8877
Epoch 00002: val_loss improved from 0.36458 to 0.31668, saving model to best-imdb-conv1d.h5
196/196 [==============================] - 3s 15ms/step - loss: 0.2763 - accuracy: 0.8876 - val_loss: 0.3167 - val_accuracy: 0.8649
Epoch 3/30
193/196 [============================>.] - ETA: 0s - loss: 0.2035 - accuracy: 0.9205
Epoch 00003: val_loss did not improve from 0.31668
196/196 [==============================] - 3s 15ms/step - loss: 0.2035 - accuracy: 0.9204 - val_loss: 0.3324 - val_accuracy: 0.8623
Epoch 4/30
193/196 [============================>.] - ETA: 0s - loss: 0.1498 - accura

In [20]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 3ms/step - loss: 0.3144 - accuracy: 0.8664


[0.3144266903400421, 0.8664000034332275]

- Case 2) Conv1D + LSTM

In [21]:
from tensorflow.keras.layers import LSTM

In [22]:
model2 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                      

In [23]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [24]:
hist2 = model2.fit(
    X_train, y_train, epochs=30, batch_size=128, validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
195/196 [============================>.] - ETA: 0s - loss: 0.4363 - accuracy: 0.7761
Epoch 00001: val_loss improved from inf to 0.29052, saving model to best-imdb-conv1d-lstm.h5
196/196 [==============================] - 27s 25ms/step - loss: 0.4363 - accuracy: 0.7762 - val_loss: 0.2905 - val_accuracy: 0.8790
Epoch 2/30
196/196 [==============================] - ETA: 0s - loss: 0.2179 - accuracy: 0.9148
Epoch 00002: val_loss improved from 0.29052 to 0.28820, saving model to best-imdb-conv1d-lstm.h5
196/196 [==============================] - 5s 23ms/step - loss: 0.2179 - accuracy: 0.9148 - val_loss: 0.2882 - val_accuracy: 0.8758
Epoch 3/30
196/196 [==============================] - ETA: 0s - loss: 0.1633 - accuracy: 0.9405
Epoch 00003: val_loss did not improve from 0.28820
196/196 [==============================] - 4s 22ms/step - loss: 0.1633 - accuracy: 0.9405 - val_loss: 0.3289 - val_accuracy: 0.8781
Epoch 4/30
196/196 [==============================] - ETA: 0s - loss: 0.12

In [25]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 4s 7ms/step - loss: 0.2901 - accuracy: 0.8786


[0.2900741994380951, 0.878600001335144]

- Case 3) Conv1D + Dense

In [26]:
model3 = Sequential([
    Embedding(10000, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(4),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_3 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 124, 64)          0         
 1D)                                                             
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 100)              

In [28]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best-imdb-conv1d-fcn.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [29]:
hist3 = model3.fit(
    X_train, y_train, epochs=30, batch_size=128, validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
192/196 [============================>.] - ETA: 0s - loss: 0.5608 - accuracy: 0.6930
Epoch 00001: val_loss improved from inf to 0.35085, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 4s 15ms/step - loss: 0.5584 - accuracy: 0.6944 - val_loss: 0.3509 - val_accuracy: 0.8477
Epoch 2/30
192/196 [============================>.] - ETA: 0s - loss: 0.2909 - accuracy: 0.8758
Epoch 00002: val_loss improved from 0.35085 to 0.26982, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 3s 14ms/step - loss: 0.2911 - accuracy: 0.8760 - val_loss: 0.2698 - val_accuracy: 0.8853
Epoch 3/30
192/196 [============================>.] - ETA: 0s - loss: 0.1989 - accuracy: 0.9222
Epoch 00003: val_loss improved from 0.26982 to 0.25932, saving model to best-imdb-conv1d-fcn.h5
196/196 [==============================] - 3s 14ms/step - loss: 0.1994 - accuracy: 0.9222 - val_loss: 0.2593 - val_accuracy: 0.8966
Epoch 4/30
194/196 [============

In [30]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 3ms/step - loss: 0.2623 - accuracy: 0.8931


[0.26226556301116943, 0.8931333422660828]